In [33]:
import pandas as pd

# Path to the Excel file
excel_file_path = "HINDALCO_1D.xlsx"

# Read the Excel file into a DataFrame
df = pd.read_excel(excel_file_path)

print(df)

       datetime   close    high     low    open   volume instrument
0    2014-01-24  114.00  115.35  113.00  113.15  5737135   HINDALCO
1    2014-01-27  111.10  112.70  109.30  112.00  8724577   HINDALCO
2    2014-01-28  113.80  115.00  109.75  110.00  4513345   HINDALCO
3    2014-01-29  111.75  114.75  111.15  114.50  4713458   HINDALCO
4    2014-01-30  108.10  110.70  107.60  110.20  5077231   HINDALCO
...         ...     ...     ...     ...     ...      ...        ...
1210 2018-12-21  224.00  227.45  221.80  224.25  6933691   HINDALCO
1211 2018-12-24  217.85  224.00  216.95  224.00  4667022   HINDALCO
1212 2018-12-26  218.60  219.80  212.90  213.85  6554865   HINDALCO
1213 2018-12-27  220.60  223.90  219.60  221.40  7947144   HINDALCO
1214 2018-12-28  223.50  226.05  221.00  221.00  5074920   HINDALCO

[1215 rows x 7 columns]


In [34]:
import pymysql

# Connect to the MySQL server
cnx = pymysql.connect(
    host='localhost',
    user='root',
    password='',
)

# Create a cursor object to execute SQL queries
cursor = cnx.cursor()

In [35]:
def createDatabase(database_name):
    
    # SQL query to create the database
    create_database_query = f'CREATE DATABASE {database_name}'

    # Execute the query to create the database
    cursor.execute(create_database_query)
    cursor.close()
    cnx.close()

    # Connect to the newly created database
    

In [36]:
def connectToDatabase(database_name):
    global cnx
    global cursor
    cnx = pymysql.connect(
        host='localhost',
        user='root',
        password='',
        database=database_name
    )
    cursor = cnx.cursor()

In [37]:
def createTable(table_name):
    create_table_query = '''
    CREATE TABLE %s (
    datetime DATETIME,
    close DECIMAL(10, 2),
    open DECIMAL(10, 2),
    high DECIMAL(10, 2),
    low DECIMAL(10, 2),
    volume INT,
    instrument VARCHAR(255)
    );
    
    ''' % (table_name)

    # Execute the query to create the table
    cursor.execute(create_table_query)

    # Commit the changes to the database
    cnx.commit()
    #print(create_table_query)

In [38]:
def checkIfDatabaseExists(database_name):


    # SQL query to fetch the list of databases
    show_databases_query = 'SHOW DATABASES'

    # Execute the query
    cursor.execute(show_databases_query)

    # Fetch all the databases
    databases = cursor.fetchall()

    # Check if the database exists
    if (database_name,) in databases:
        return True
    else:
        return False


In [39]:
def checkIfTableExists(table_name):


    # SQL query to check if the table exists
    check_table_query = f"SHOW TABLES LIKE '{table_name}'"

    # Execute the query
    cursor.execute(check_table_query)

    # Fetch the result
    result = cursor.fetchone()

    # Check if the table exists
    if result:
        return True
    else:
        return False




In [40]:
if(checkIfDatabaseExists("invsto")):
    print("Database already exists")
else:
    createDatabase("invsto")

Database already exists


In [41]:
connectToDatabase("invsto")

In [42]:
# Close the cursor and connection
if(checkIfTableExists("invsto_hindalco_details")):
    print("Table already exists")
else:
    createTable("invsto_hindalco_details")


Table already exists


In [43]:
def insertValuesInTable(datetime,close,open_price,high,low,volume,instrument):

    sql = """INSERT INTO invsto_hindalco_details (datetime, close, open, high, low, volume, instrument)
    VALUES (%s, %f, %f, %f, %f, %d, %s)"""

    # Insert the values into the table
    sql = "INSERT INTO invsto_hindalco_details (datetime, close, open, high, low, volume, instrument) VALUES (%s, %s, %s, %s, %s, %s, %s)"
    cursor.execute(sql, (datetime, close, open_price, high, low, volume, 'HINDALCO'))

    # Commit the changes to the database
    cnx.commit()

In [14]:
def addDftoTable():
    
    for i in range(0,len(df.index)):

        row_wise_datetime = df.loc[i]["datetime"]
        row_wise_close = df.loc[i]["close"]
        row_wise_high = df.loc[i]["high"]
        row_wise_low = df.loc[i]["low"]
        row_wise_open = df.loc[i]["open"]
        row_wise_volume = df.loc[i]["volume"]
        row_wise_instrument = df.loc[i]["instrument"]
        insertValuesInTable(row_wise_datetime,row_wise_close,row_wise_open,row_wise_high,row_wise_low,row_wise_volume,row_wise_instrument)


    
    


In [44]:
import pandas as pd

# Calculate the 10-day and 20-day SMAs
df['SMA10'] = df['close'].rolling(window=10).mean()
df['SMA20'] = df['close'].rolling(window=20).mean()

# Initialize a column to store trading signals
df['Signal'] = 0

# Generate trading signals
df.loc[(df['SMA10'] > df['SMA20']) & (df['SMA10'].shift(1) < df['SMA20'].shift(1)), 'Signal'] = 1  # Buy signal
df.loc[(df['SMA10'] < df['SMA20']) & (df['SMA10'].shift(1) > df['SMA20'].shift(1)), 'Signal'] = -1  # Sell signal

# Calculate the positions based on the trading signals
df['Position'] = df['Signal'].diff()

# Backtest the trading strategy
initial_capital = 10000  # Assuming initial capital of 10000
position = 0  # 0 for out of market, 1 for long, -1 for short
shares = 0  # Number of shares held
for i, row in df.iterrows():
    if row['Position'] == 1:  # Buy signal
        if position == 0:  # If out of market
            position = 1  # Enter long position
            shares = int(initial_capital / row['close'])
            initial_capital = 0
            print(f"Buy {shares} shares at {row['close']:.2f}")
    elif row['Position'] == -1:  # Sell signal
        if position == 1:  # If in long position
            position = 0  # Exit long position
            initial_capital = shares * row['close']
            pl = initial_capital - 10000  # Calculate profit or loss
            print(f"Sell {shares} shares at {row['close']:.2f} | P/L: {pl:.2f}")
            shares = 0


# Calculate the final portfolio value
final_value = initial_capital + (shares * df['close'].iloc[-1])
pl_final = final_value - 10000  # Calculate final profit or loss
print(f"Final portfolio value: {final_value:.2f} | P/L: {pl_final:.2f}")


Buy 82 shares at 120.80
Sell 82 shares at 124.55 | P/L: 213.10
Buy 74 shares at 136.75
Sell 74 shares at 152.05 | P/L: 1251.70
Buy 62 shares at 178.60
Sell 62 shares at 156.50 | P/L: -297.00
Buy 63 shares at 153.20
Sell 63 shares at 160.10 | P/L: 86.30
Buy 71 shares at 141.90
Sell 71 shares at 149.20 | P/L: 593.20
Buy 81 shares at 130.35
Sell 81 shares at 139.90 | P/L: 1331.90
Buy 86 shares at 130.60
Sell 86 shares at 133.50 | P/L: 1481.00
Buy 89 shares at 128.35
Sell 89 shares at 108.70 | P/L: -325.70
Buy 88 shares at 109.55
Sell 88 shares at 86.15 | P/L: -2418.80
Buy 92 shares at 81.85
Sell 92 shares at 75.70 | P/L: -3035.60
Buy 90 shares at 76.70
Sell 90 shares at 70.25 | P/L: -3677.50
Buy 69 shares at 91.60
Sell 69 shares at 103.10 | P/L: -2886.10
Buy 48 shares at 145.55
Sell 48 shares at 154.25 | P/L: -2596.00
Buy 49 shares at 150.50
Sell 49 shares at 167.40 | P/L: -1797.40
Buy 48 shares at 168.80
Sell 48 shares at 172.25 | P/L: -1732.00
Buy 45 shares at 183.65
Sell 45 shares at 1

In [45]:
print(df)

       datetime   close    high     low    open   volume instrument    SMA10  \
0    2014-01-24  114.00  115.35  113.00  113.15  5737135   HINDALCO      NaN   
1    2014-01-27  111.10  112.70  109.30  112.00  8724577   HINDALCO      NaN   
2    2014-01-28  113.80  115.00  109.75  110.00  4513345   HINDALCO      NaN   
3    2014-01-29  111.75  114.75  111.15  114.50  4713458   HINDALCO      NaN   
4    2014-01-30  108.10  110.70  107.60  110.20  5077231   HINDALCO      NaN   
...         ...     ...     ...     ...     ...      ...        ...      ...   
1210 2018-12-21  224.00  227.45  221.80  224.25  6933691   HINDALCO  222.165   
1211 2018-12-24  217.85  224.00  216.95  224.00  4667022   HINDALCO  222.160   
1212 2018-12-26  218.60  219.80  212.90  213.85  6554865   HINDALCO  222.395   
1213 2018-12-27  220.60  223.90  219.60  221.40  7947144   HINDALCO  222.325   
1214 2018-12-28  223.50  226.05  221.00  221.00  5074920   HINDALCO  222.645   

         SMA20  Signal  Position  
0   

In [46]:
#Unit Test



In [47]:
cursor.close()
cnx.close()